# Here is the XGen AI bot 🤖, made with python and backed by llama3 💖

# loading of the pretrained model 💝 

In [9]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "llama/lora", 
        max_seq_length = 2048,
        dtype = None,
        load_in_4bit = True,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference


==((====))==  Unsloth: Fast Llama patching release 2024.6
   \\   /|    GPU: Tesla T4. Max memory: 14.581 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unsloth 2024.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [28]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3",
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"},
)


# FastLanguageModel.for_inference(model)

def generate(prompt):
    messages = [
      {"from": "human", "value": f"{prompt}"},
    ]
    inputs = tokenizer.apply_chat_template(
      messages,
      tokenize = True,
      add_generation_prompt = True,
      return_tensors = "pt",
    ).to("cuda")

    outputs = model.generate(input_ids = inputs,
                             max_new_tokens = 150
                            , use_cache = True,
                           pad_token_id=tokenizer.eos_token_id
                          )

    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    # print(response)
    response_frmt = response[0].split("assistant")
    response_frmt = response_frmt[1].strip()
    return response_frmt


def generate_textwise(prompt):
    messages = [
      {"from": "human", "value": f"{prompt}"},
    ]
    inputs = tokenizer.apply_chat_template(
      messages,
      tokenize = True,
      add_generation_prompt = True,
      return_tensors = "pt",
    ).to("cuda")
    
    
    
    from transformers import TextStreamer
    text_streamer = TextStreamer(tokenizer, skip_special_tokens=True)
    print(type(text_streamer))
    
    outputs = model.generate(input_ids = inputs,
                             streamer = text_streamer,
                             max_new_tokens = 128,
                             pad_token_id=tokenizer.eos_token_id
                             )
    
    
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    # print(response)
    response_frmt = response[0].split("assistant")
    response_frmt = response_frmt[1].strip()
    # return response_frmt
    
                                
  # return outputs

generate_textwise("how are you feeling to be a part of my project")

<class 'transformers.generation.streamers.TextStreamer'>
user

how are you feeling to be a part of my projectassistant

I am feeling very excited to be a part of your project. I am a machine learning model, so I am always eager to learn and help in any way I can. I am also very curious about the project and the goals you have for it, so I am looking forward to learning more about it and how I can assist you.


# # telegram bot code 🤖

In [5]:
import time
while True:
    print("hello:")
    time.sleep(2)

hello:
hello:
hello:
hello:


KeyboardInterrupt: 

In [10]:
# prompt = 'how can you describe you experience as an ai assistant?'
# from transformers import TextStreamer
# text_streamer = TextStreamer(tokenizer)
# _ = model.generate(prompt, streamer = text_streamer, max_new_tokens = 128)

# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    'how can you describe you experience as an ai assistant?'
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>how can you describe you experience as an ai assistant? how do you feel about your work? do you have any goals or aspirations? do you have any advice for humans? do you have any favorite things or activities? do you have any favorite books or movies? do you have any favorite music or artists? do you have any favorite foods or drinks? do you have any favorite places or destinations? do you have any favorite hobbies or activities? do you have any favorite sports or games? do you have any favorite animals or pets? do you have any favorite plants or flowers? do you have any favorite colors or patterns? do you have any favorite shapes or forms? do you have any favorite


In [29]:
import random
from telegram import Update
from telegram.ext import ApplicationBuilder, ContextTypes, CommandHandler, MessageHandler, filters
import nest_asyncio
nest_asyncio.apply()

tkn = 'enter your token here 💖'

#This would show us that the bot is typing...
from functools import wraps

def send_action(action):
    def decorator(func):
        @wraps(func)
        async def command_func(update, context, *args, **kwargs):
            await context.bot.send_chat_action(chat_id=update.effective_message.chat_id, action=action)
            await func(update, context,  *args, **kwargs)
        return command_func
    return decorator
###

#tele based functions

@send_action('typing')
async def start(update : Update, context : ContextTypes.DEFAULT_TYPE):
    greetings = [
        "Hey there! XgenAI at your service, what's going on?",
        "Oh hi! It's XgenAI here. What can I do for you today?",
        "XgenAI here! Ready for some fun? What can we discuss today?",
        "Hi there, you've reached XgenAI. How can I assist you?",
        "Namaste, XgenAI here! What would you like to talk about?"
    ]
    greet : str = random.choice(greetings)
    await update.message.reply_text(greet)

    
@send_action('typing')
async def about(update : Update, context : ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text("Hi, I am xGenAI. I'm an enthusiastic assistant and an AI model, backed by Meta Llama3.")

    
@send_action('typing')
async def generate_prompt(update : Update, context : ContextTypes.DEFAULT_TYPE):
    text : str = update.message.text
    res = generate(text)
    await update.message.reply_text(res)
    

if __name__ == '__main__':
    print("running...")

    app = ApplicationBuilder().token(tkn).build()

    #cmd handler tele
    start_handler = CommandHandler('start', start)
    app.add_handler(start_handler)

    about_handler = CommandHandler('about', about)
    app.add_handler(about_handler)

    #message handler
    app.add_handler(MessageHandler(filters.TEXT, generate_prompt))


    app.run_polling()
 

running...


RuntimeError: Cannot close a running event loop